### Importing libraries

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### Getting data from the wikipedia page

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
doc = requests.get(url).text
soup = BeautifulSoup(doc, 'html.parser')

table = soup.find('tbody')
rows = table.select('tr')
row = [r.get_text() for r in rows]

### Create the dataframe

In [49]:
df = pd.DataFrame(row)
df = df[0].str.split('\n',expand = True)
df = df.rename(columns = df[0] )
df.columns = df.iloc[0]
df = df.drop(index = 0)
df.head()

,,Postal Code,,Borough,,Neighborhood,
1,,M1A,,Not assigned,,Not assigned,
2,,M2A,,Not assigned,,Not assigned,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",


### Ignore rows which have 'Not Assigned'

In [59]:
df = df[df.Borough!='Not assigned']
df.head()

,,Postal Code,,Borough,,Neighborhood,
3,,M3A,,North York,,Parkwoods,
4,,M4A,,North York,,Victoria Village,
5,,M5A,,Downtown Toronto,,"Regent Park, Harbourfront",
6,,M6A,,North York,,"Lawrence Manor, Lawrence Heights",
7,,M7A,,Downtown Toronto,,"Queen's Park, Ontario Provincial Government",


### Replacing 'Not assigned' Neighborhood with Borough name

In [57]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood']  = df['Borough']

### Same postcode Neighborhoods

In [64]:
df = df.groupby(['Postal Code', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [67]:
df.shape

(103, 3)

In [70]:
df.to_csv('toronto_data', index = False)